In [1]:
import os
import sys

# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive
    import tensorflow as tf
    print("Versión de TensorFlow:", tf.__version__)
    print("Dispositivos disponibles:", tf.config.list_physical_devices())

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/DATASET TFM'

    path_workspace = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)

    # Verificar la GPU
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        raise SystemError('GPU no encontrada')
    print(f'Encontrada GPU: {device_name}')

    # Habilitar la GPU para TensorFlow
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print('Memoria de la GPU configurada dinámicamente')
    else:
        print('No se encontraron dispositivos GPU configurables')
else:
    print("El código se está ejecutando en un entorno local.")
    path_workspace ='C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/'
    path_absolute = os.getcwd().replace("\\", "/")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/TFM/WorkSpace/DATASET TFM'

datasets_path = "/ml-25m/"
path_absolute = path_absolute+datasets_path


sys.path.append(path_workspace)

El código se está ejecutando en un entorno local.


El notebook realiza las siguientes operaciones con el objetivo de preparar y analizar un conjunto de datos de películas y sus calificaciones, incorporando etiquetas de géneros y tags proporcionados tanto por un sistema de etiquetado genómico como por los usuarios:

1. **Carga de Datos**: Importa dos archivos CSV (`ratings.csv` y `movies.csv`) que contienen datos de calificaciones de usuarios y detalles de películas, respectivamente.

2. **Exploración Inicial**:
   - Muestra las dimensiones y las primeras filas de ambos conjuntos de datos para entender su estructura.
   - Calcula y muestra el número de valores únicos para `movieId` y `title` en el conjunto de datos de películas, y para `userId` y `movieId` en el conjunto de datos de calificaciones.

3. **Fusión de Datos**:
   - Combina (`merge`) los datos de calificaciones y películas en un solo DataFrame basado en `movieId`, para relacionar cada calificación con la película correspondiente.
   - Vuelve a calcular el número de valores únicos para `userId`, `movieId`, y `title` después de la fusión para asegurar la integridad de los datos.

4. **Limpieza de Datos**:
   - Verifica y muestra la cantidad de valores nulos en el DataFrame combinado.
   - Importa un tercer conjunto de datos (`movies_genome_tags.csv`) que contiene tags genómicos para las películas, y fusiona este conjunto con el DataFrame anterior, añadiendo la información de tags donde sea aplicable.
   - Maneja los valores nulos y las duplicaciones, especialmente enfocándose en las columnas de tags genómicos y tags proporcionados por los usuarios.

5. **Análisis de Tags**:
   - Carga un cuarto conjunto de datos (`movies_tags_by_users.csv`) que incluye tags asignados por los usuarios a las películas.
   - Realiza una nueva fusión para combinar los tags de usuarios con el DataFrame previamente fusionado, creando un conjunto de datos ampliado que incluye tanto los tags genómicos como los de los usuarios.
   - Analiza la cantidad de valores únicos y la presencia de valores nulos tras esta última fusión, destacando la cantidad significativa de valores nulos en las columnas de tags.

6. **Preparación Final de Datos**:
   - Reordena las columnas del DataFrame final para mejorar la organización y facilitar análisis futuros.
   - Convierte la columna `genres` en un formato de codificación one-hot, creando una columna separada para cada género y marcando con un 1 la presencia de dicho género en cada película.
   - Elimina la columna original `genres` y ajusta el orden de las columnas para colocar las variables objetivo (`rating` y `timestamp`) en posiciones específicas.

7. **Exportación de Datos**:
   - Guarda el DataFrame final, ahora con géneros codificados de manera one-hot y tags de películas tanto genómicos como de usuarios, a un nuevo archivo CSV para su uso en análisis o modelado posterior.

Este proceso detallado prepara el conjunto de datos para análisis o modelado más complejos, asegurando que la información de películas, calificaciones, y tags esté limpia, organizada, y rica en características.

Se ha elejido los 5 tags mas relevantes por pelicula y se ha fusionado con el conjunto de datos de genome tags para tener una clumna con los 5 tas más relevantes por pelicula. 

In [2]:
import pandas as pd
import Utils.utils as util

In [3]:
ratings= pd.read_csv(path_absolute+'ratings.csv')
movies= pd.read_csv(path_absolute+'movies.csv')

In [4]:
print(ratings.shape)
ratings.head()


(25000095, 4)


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [5]:
print(movies.shape)
movies.head()


(62423, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
num_unique_movieId = movies['movieId'].nunique()
num_unique_title = movies['title'].nunique()

print("Número de valores únicos en la columna movieId:", num_unique_movieId)
print("Número de valores únicos en la columna title:", num_unique_title)

Número de valores únicos en la columna movieId: 62423
Número de valores únicos en la columna title: 62325


In [7]:
# Contar el número de valores únicos en la columna movieId
num_unique_userID = ratings['userId'].nunique()
num_unique_movieId = ratings['movieId'].nunique()

print("Número de valores únicos en la columna userId:", num_unique_userID)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna userId: 162541
Número de valores únicos en la columna movieId: 59047


In [8]:
merged_df_ratings_movies = pd.merge(ratings, movies, on='movieId', suffixes=('_df1', '_df2'))

In [9]:
print(merged_df_ratings_movies.shape)
merged_df_ratings_movies.head()

(25000095, 6)


,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller


In [10]:
# Contar el número de valores únicos en la columna movieId
num_unique_userID = merged_df_ratings_movies['userId'].nunique()
num_unique_movieId = merged_df_ratings_movies['movieId'].nunique()
num_unique_title = merged_df_ratings_movies['title'].nunique()

print("Número de valores únicos en la columna userId:", num_unique_userID)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)
print("Número de valores únicos en la columna titte:", num_unique_title)

Número de valores únicos en la columna userId: 162541
Número de valores únicos en la columna movieId: 59047
Número de valores únicos en la columna titte: 58958


In [11]:
merged_df_ratings_movies_non_duplicated = merged_df_ratings_movies[~merged_df_ratings_movies.duplicated(subset=['userId', 'movieId'], keep='first')]


In [12]:
print(merged_df_ratings_movies_non_duplicated.shape)
merged_df_ratings_movies_non_duplicated.head()

(25000095, 6)


,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller


In [13]:
# Contar el número de valores únicos en la columna movieId
num_unique_userID = merged_df_ratings_movies_non_duplicated['userId'].nunique()
num_unique_movieId = merged_df_ratings_movies_non_duplicated['movieId'].nunique()
num_unique_title = merged_df_ratings_movies_non_duplicated['title'].nunique()

print("Número de valores únicos en la columna userId:", num_unique_userID)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)
print("Número de valores únicos en la columna titte:", num_unique_title)

Número de valores únicos en la columna userId: 162541
Número de valores únicos en la columna movieId: 59047
Número de valores únicos en la columna titte: 58958


In [14]:
# Cuenta los valores nulos en cada columna del DataFrame
valores_nulos_por_columna = merged_df_ratings_movies_non_duplicated.isnull().sum()
# Cuenta los valores nulos en todo el DataFrame
total_valores_nulos = merged_df_ratings_movies_non_duplicated.isnull().sum().sum()
# Imprime la cantidad de valores nulos por columna
print("Valores nulos por columna:")
print(valores_nulos_por_columna)
# Imprime el total de valores nulos en el DataFrame
print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

Valores nulos por columna:
userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64

Total de valores nulos en el dataset: 0


In [15]:
df_movies_genome_tags= pd.read_csv(path_absolute+'movies_genome_tags.csv')

In [16]:
print(df_movies_genome_tags.shape)
df_movies_genome_tags.head()

(13816, 2)


,movieId,tag
0,1,"animation, kids and family, pixar animation, t..."
1,2,"children, adventure, fantasy, kids, special ef..."
2,3,"comedy, original, sequel, sequels, good sequel"
3,4,"women, girlie movie, divorce, romantic, chick ..."
4,5,"father daughter relationship, sequel, sequels,..."


In [17]:
# Contar el número de valores únicos en la columna movieId
num_unique_movieId = df_movies_genome_tags['movieId'].nunique()
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna movieId: 13816


In [18]:
merged_df_ratings_movies_genomeTag = pd.merge(merged_df_ratings_movies_non_duplicated, df_movies_genome_tags, on='movieId', how='left', suffixes=('_df1', '_df2'))

In [19]:
print(merged_df_ratings_movies_genomeTag.shape)
merged_df_ratings_movies_genomeTag.head()

(25000095, 7)


,userId,movieId,rating,timestamp,title,genres,tag
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar..."
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar..."
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar..."
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar..."
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar..."


In [20]:
# Contar el número de valores únicos en la columna movieId
num_unique_userID = merged_df_ratings_movies_genomeTag['userId'].nunique()
num_unique_movieId = merged_df_ratings_movies_genomeTag['movieId'].nunique()

print("Número de valores únicos en la columna userId:", num_unique_userID)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna userId: 162541
Número de valores únicos en la columna movieId: 59047


In [21]:
# Cuenta los valores nulos en cada columna del DataFrame
valores_nulos_por_columna = merged_df_ratings_movies_genomeTag.isnull().sum()
# Cuenta los valores nulos en todo el DataFrame
total_valores_nulos = merged_df_ratings_movies_genomeTag.isnull().sum().sum()
# Imprime la cantidad de valores nulos por columna
print("Valores nulos por columna:")
print(valores_nulos_por_columna)
# Imprime el total de valores nulos en el DataFrame
print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

Valores nulos por columna:
userId            0
movieId           0
rating            0
timestamp         0
title             0
genres            0
tag          325982
dtype: int64

Total de valores nulos en el dataset: 325982


Hay 325982 usurio-pelicula que no tienen etiqueta genomica.

In [22]:
merged_df_ratings_movies_genomeTag_nonduplicated = merged_df_ratings_movies_genomeTag[~merged_df_ratings_movies_genomeTag.duplicated(
    subset=['userId', 'movieId'], keep='first')]


In [23]:
print(merged_df_ratings_movies_genomeTag_nonduplicated.shape)
merged_df_ratings_movies_genomeTag_nonduplicated.head()

(25000095, 7)


,userId,movieId,rating,timestamp,title,genres,tag
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar..."
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar..."
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar..."
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar..."
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar..."


In [24]:
# Contar el número de valores únicos en la columna movieId
num_unique_userID = merged_df_ratings_movies_genomeTag_nonduplicated['userId'].nunique()
num_unique_movieId = merged_df_ratings_movies_genomeTag_nonduplicated['movieId'].nunique()

print("Número de valores únicos en la columna userId:", num_unique_userID)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna userId: 162541
Número de valores únicos en la columna movieId: 59047


In [25]:
df_movies_tags_by_user= pd.read_csv(path_absolute+'movies_tags_by_users.csv')

In [26]:
print(df_movies_tags_by_user.shape)
df_movies_tags_by_user.head()

(305356, 3)


,userId,movieId,tag
0,3,260,", conflict, Special Effects, underdog story, N..."
1,4,1732,"drugs, Best Movie Ever, girls, Nudity (Full Fr..."
2,4,7569,"london england, misogyny, gyrocopter, secret b..."
3,4,44665,"KAF, To See, hitman, gangster, Rythm, Josh Har..."
4,4,115569,"television, thrilling, American Dream, depress..."


In [27]:
# Contar el número de valores únicos en la columna movieId
num_unique_userID = df_movies_tags_by_user['userId'].nunique()
num_unique_movieId = df_movies_tags_by_user['movieId'].nunique()

print("Número de valores únicos en la columna userId:", num_unique_userID)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna userId: 14592
Número de valores únicos en la columna movieId: 45251


In [28]:
# df_filter = df_movies_tags_by_user[(df_movies_tags_by_user['userId'] == 1) & (df_movies_tags_by_user['movieId'] == 44665)]
# df_filter = df_movies_tags_by_user[(df_movies_tags_by_user['userId'] == 1)]
# df_filter.head()


In [29]:
merged_df_movies_tagGenome_tag_Users = pd.merge(merged_df_ratings_movies_genomeTag_nonduplicated, df_movies_tags_by_user, on=['userId', 'movieId'], how='left', suffixes=('_genome', '_by_user'))

In [30]:
print(merged_df_movies_tagGenome_tag_Users.shape)
merged_df_movies_tagGenome_tag_Users.head()

(25000095, 8)


,userId,movieId,rating,timestamp,title,genres,tag_genome,tag_by_user
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar...",NaN
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar...",NaN
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar...",NaN
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar...",NaN
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar...",NaN


In [31]:
# Contar el número de valores únicos en la columna movieId
num_unique_userID = merged_df_movies_tagGenome_tag_Users['userId'].nunique()
num_unique_movieId = merged_df_movies_tagGenome_tag_Users['movieId'].nunique()

print("Número de valores únicos en la columna userId:", num_unique_userID)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna userId: 162541
Número de valores únicos en la columna movieId: 59047


In [32]:
# Cuenta los valores nulos en cada columna del DataFrame
valores_nulos_por_columna = merged_df_movies_tagGenome_tag_Users.isnull().sum()
# Cuenta los valores nulos en todo el DataFrame
total_valores_nulos = merged_df_movies_tagGenome_tag_Users.isnull().sum().sum()
# Imprime la cantidad de valores nulos por columna
print("Valores nulos por columna:")
print(valores_nulos_por_columna)
# Imprime el total de valores nulos en el DataFrame
print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

Valores nulos por columna:
userId                0
movieId               0
rating                0
timestamp             0
title                 0
genres                0
tag_genome       325982
tag_by_user    24789370
dtype: int64

Total de valores nulos en el dataset: 25115352


Hay 325982 usurio-pelicula que no tienen etiqueta genomica.

Hay 24789370 usuario pelicula que no tienen etiqueta por usuario.

In [33]:
merged_df_movies_tagGenome_tag_Users_non_duplicated = merged_df_movies_tagGenome_tag_Users[~merged_df_movies_tagGenome_tag_Users.duplicated(
    subset=['userId', 'movieId'], keep='first')]

In [34]:
filas_nulas = merged_df_movies_tagGenome_tag_Users_non_duplicated[merged_df_movies_tagGenome_tag_Users_non_duplicated[['tag_genome', 'tag_by_user']].isnull().all(axis=1)]
print(filas_nulas.shape)
filas_nulas.head()

(312968, 8)


,userId,movieId,rating,timestamp,title,genres,tag_genome,tag_by_user
590201,1,8685,1.0,1147878023,"Miracle of Marcelino, The (Marcelino pan y vin...",Comedy|Drama,NaN,NaN
590202,3821,8685,3.5,1113598322,"Miracle of Marcelino, The (Marcelino pan y vin...",Comedy|Drama,NaN,NaN
590203,6730,8685,4.0,1382959197,"Miracle of Marcelino, The (Marcelino pan y vin...",Comedy|Drama,NaN,NaN
590204,14175,8685,3.5,1171730449,"Miracle of Marcelino, The (Marcelino pan y vin...",Comedy|Drama,NaN,NaN
590205,20055,8685,3.0,1189414009,"Miracle of Marcelino, The (Marcelino pan y vin...",Comedy|Drama,NaN,NaN


In [35]:
print(merged_df_movies_tagGenome_tag_Users_non_duplicated.shape)
merged_df_movies_tagGenome_tag_Users_non_duplicated.head()

(25000095, 8)


,userId,movieId,rating,timestamp,title,genres,tag_genome,tag_by_user
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar...",NaN
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar...",NaN
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar...",NaN
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar...",NaN
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,"masterpiece, hit men, gratuitous violence, dar...",NaN


In [36]:
# Contar el número de valores únicos en la columna movieId
num_unique_userID = merged_df_movies_tagGenome_tag_Users_non_duplicated['userId'].nunique()
num_unique_movieId = merged_df_movies_tagGenome_tag_Users_non_duplicated['movieId'].nunique()

print("Número de valores únicos en la columna userId:", num_unique_userID)
print("Número de valores únicos en la columna movieId:", num_unique_movieId)

Número de valores únicos en la columna userId: 162541
Número de valores únicos en la columna movieId: 59047


In [37]:
# Cuenta los valores nulos en cada columna del DataFrame
valores_nulos_por_columna = merged_df_movies_tagGenome_tag_Users_non_duplicated.isnull().sum()
# Cuenta los valores nulos en todo el DataFrame
total_valores_nulos = merged_df_movies_tagGenome_tag_Users_non_duplicated.isnull().sum().sum()
# Imprime la cantidad de valores nulos por columna
print("Valores nulos por columna:")
print(valores_nulos_por_columna)
# Imprime el total de valores nulos en el DataFrame
print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

Valores nulos por columna:
userId                0
movieId               0
rating                0
timestamp             0
title                 0
genres                0
tag_genome       325982
tag_by_user    24789370
dtype: int64

Total de valores nulos en el dataset: 25115352


In [38]:
# Lista con el nuevo orden de las columnas
new_column_order = ['userId', 'movieId', 'tag_by_user', 'tag_genome', 'title', 'genres', 'timestamp', 'rating']
# Reordenar las columnas
merged_df_movies_tagGenome_tag_Users_non_duplicated = merged_df_movies_tagGenome_tag_Users_non_duplicated.reindex(columns=new_column_order)


In [39]:
print(merged_df_movies_tagGenome_tag_Users_non_duplicated.shape)
merged_df_movies_tagGenome_tag_Users_non_duplicated.head()

(25000095, 8)


,userId,movieId,tag_by_user,tag_genome,title,genres,timestamp,rating
0,1,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1147880044,5.0
1,3,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1439474476,5.0
2,4,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1573938898,4.0
3,5,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,830786155,4.0
4,7,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,835444730,4.0


In [40]:
# merged_df_movies_tagGenome_tag_Users_non_duplicated.to_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs.csv', index=False)

-----------------------------------

In [41]:
# Especificar el tipo de datos de la columna como cadena (string)
dataFrame = pd.read_csv(path_absolute + 'data_moviesRating_tgGen_tgUsrs.csv', dtype=str)


In [42]:
# Divide la columna 'géneros' en múltiples columnas usando '|' como delimitador y convierte en binario (0 o 1)
generos_separados = dataFrame['genres'].str.get_dummies(sep='|')

# Concatena los nuevos géneros_separados con el DataFrame original
df_full_tag_genome_byuser_onehot_encode = pd.concat([dataFrame, generos_separados], axis=1)

# Elimina la columna original 'géneros' si ya no la necesitas
df_full_tag_genome_byuser_onehot_encode = df_full_tag_genome_byuser_onehot_encode.drop(columns=['genres'])

In [43]:
print(df_full_tag_genome_byuser_onehot_encode.shape)
df_full_tag_genome_byuser_onehot_encode.head()

(25000095, 27)


,userId,movieId,tag_by_user,tag_genome,title,timestamp,rating,(no genres listed),Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),1147880044,5.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),1439474476,5.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),1573938898,4.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,5,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),830786155,4.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,7,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),835444730,4.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [44]:
df_aux = df_full_tag_genome_byuser_onehot_encode.copy()

In [45]:
columna_a_mover = df_aux.pop('rating')
# Inserta la columna 'rating' en la última posición del DataFrame
df_aux.insert(len(df_aux.columns), 'rating', columna_a_mover)

columna_a_mover = df_aux.pop('timestamp')
# Inserta la columna 'rating' en la última posición del DataFrame
df_aux.insert(len(df_aux.columns)-1, 'timestamp', columna_a_mover)
data_moviesRating_tgGen_tgUsrs_onehot_encode = df_aux.copy()

In [46]:
data_moviesRating_tgGen_tgUsrs_onehot_encode = data_moviesRating_tgGen_tgUsrs_onehot_encode.drop(columns=['(no genres listed)'])

In [47]:
print(data_moviesRating_tgGen_tgUsrs_onehot_encode.shape)
data_moviesRating_tgGen_tgUsrs_onehot_encode.head()

(25000095, 26)


,userId,movieId,tag_by_user,tag_genome,title,Action,Adventure,Animation,Children,Comedy,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,timestamp,rating
0,1,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),0,0,0,0,1,...,0,0,0,0,0,1,0,0,1147880044,5.0
1,3,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),0,0,0,0,1,...,0,0,0,0,0,1,0,0,1439474476,5.0
2,4,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),0,0,0,0,1,...,0,0,0,0,0,1,0,0,1573938898,4.0
3,5,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),0,0,0,0,1,...,0,0,0,0,0,1,0,0,830786155,4.0
4,7,296,NaN,"masterpiece, hit men, gratuitous violence, dar...",Pulp Fiction (1994),0,0,0,0,1,...,0,0,0,0,0,1,0,0,835444730,4.0


In [48]:
# data_moviesRating_tgGen_tgUsrs_onehot_encode.to_csv(path_absolute+'data_moviesRating_tgGen_tgUsrs_onehot_encode.csv', index=False)

---------------------------------------------------------------------------------------------------------------------